# **텍스트 정규화**

In [15]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [16]:
import pandas as pd

print('target 클래스의 값과 분포도\n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들\n', news_data.target_names)

target 클래스의 값과 분포도
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [17]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [18]:
from sklearn.datasets import fetch_20newsgroups

#학습용 데이터 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

#테스트용 데이터 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target

print(f'학습 데이터 크기: {len(train_news.data)}, 테스트 데이터 크기: {len(test_news.data)}')

학습 데이터 크기: 11314, 테스트 데이터 크기: 7532


# **피처 벡터화 변환 & 모델 학습/예측/평가**

In [19]:
#카운트 기반의 피처 벡터화 수행
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)

X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터의 CountVectorizer Shape: (11314, 101631)


In [20]:
#로지스틱 회귀 모델 적용
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
accuracy = accuracy_score(y_test, pred)
print(f'CountVectorized Logistic Regression의 예측 정확도: {accuracy:.3f}')

CountVectorized Logistic Regression의 예측 정확도: 0.617


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
#TF-IDF 기반의 피처 벡터화 수행
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

print('학습 데이터의 TF-IDF Vectorizer Shape:', X_train_tfidf_vect.shape)

학습 데이터의 TF-IDF Vectorizer Shape: (11314, 101631)


In [ ]:
#로지스틱 회귀 모델 적용
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
accuracy = accuracy_score(y_test, pred)
print(f'TF-IDF Logistic Regression의 예측 정확도: {accuracy:.3f}')

TF-IDF Logistic Regression의 예측 정확도: 0.678


In [ ]:
#TfidfVectorizer의 파라미터 변경
tfdif_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#로지스틱 회귀 모델 적용
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
accuracy = accuracy_score(y_test, pred)
print(f'TF-IDF Logistic Regression의 예측 정확도: {accuracy:.3f}')

TF-IDF Logistic Regression의 예측 정확도: 0.678


In [ ]:
#로지스틱 회귀 모델 하이퍼 파라미터 최적화
from sklearn.model_selection import GridSearchCV

params = {'C': [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression Best C Parameter:', grid_cv_lr.best_params_)

pred = grid_cv_lr.predict(X_test_tfidf_vect)
accuracy = accuracy_score(y_test, pred)
print(f'TF-IDF Vectorized Logistic Regression의 예측 정확도: {accuracy:.3f}')

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression Best C Parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도: 0.689


# **사이킷런 Pipeline + GridSearchCV**

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

params = {'tfidf_vect__ngram_range': [(1,1),(1,2),(1,3)],
          'tfidf_vect__max_df': [100, 300, 700],
          'lr_clf__C': [1, 5, 10]}

grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
accuracy = accuracy_score(y_test, pred)

print(f'Pipeline을 통한 Logistic Regression의 예측 정확도: {accuracy:.3f}')